In [1]:
from pyomo.environ import *

In [3]:
# instanciar o modelo
model = AbstractModel()

n = 4
# parametros
model.n = Param(initialize = n)
model.i = Param(initialize = n)
model.j = Param(initialize = n)
model.c = Param(initialize = 2) # classes
model.CT = Param(initialize = 25) # capacidade do trem

precos_ini = {(1,2,1):20,(1,3,1):45,(1,4,1):75,(2,3,1):25,(2,4,1):55,(3,4,1):30,(1,2,2):25,(1,3,2):69,(1,4,2):88,(2,3,2):32,(2,4,2):93,(3,4,2):33}
demanda_ini = {(1,2,1):3,(1,3,1):10,(1,4,1):2,(2,3,1):2,(2,4,1):1,(3,4,1):12,(1,2,2):3,(1,3,2):10,(1,4,2):2,(2,3,2):2,(2,4,2):1,(3,4,2):12}


precos = {(i,j,c): precos_ini[(i,j,c)] if (i,j,c) in precos_ini else 0  for i in range(1, n+1) for j in range(1, n+1) for c in range(1, 2+1) }
demand = {(i,j,c): demanda_ini[(i,j,c)] if (i,j,c) in demanda_ini else 0  for i in range(1, n+1) for j in range(1, n+1) for c in range(1, 2+1) }


# conjuntos
model.I = RangeSet(1, model.i)
model.J = RangeSet(1, model.j)
model.C = RangeSet(1, model.c)

#parametros indexados
model.COS = Param(model.I, model.J, model.C, initialize = precos)
model.dem = Param(model.I, model.J, model.C, initialize = demand)

# variaveis de desicao
model.X = Var(model.I, model.J, model.C, within = Integers)

# funcao objetivo
def fo(model):
    return sum( model.COS[i,j,c]*model.X[i,j,c] for i in model.I for j in model.J for c in model.C)
model.OBJ = Objective(rule = fo, sense = maximize)

# restricoes
def demanda(model, j):
    if j >= 2:
        return sum(model.X[i,i+1,c] for i in model.I for c in model.C if i<j) - sum(model.dem[i,j,c] for i in model.I for c in model.C) - sum(model.dem[i,j-1,c] for i in model.I for c in model.C)>= 0
    else:
        return Constraint.Skip
model.ConstrainDemanda = Constraint(model.I, rule = demanda)

def capacidade(model, i):
    if i < model.n:
        return sum(model.X[i,i+1,c] for c in model.C) <= model.CT
    else:
        return Constraint.Skip
model.ContrainCapasidade = Constraint(model.I, rule = capacidade)



instance = model.create_instance()

opt = SolverFactory('cplex')

results = opt.solve(instance, tee=True)

# instance.pprint()
print(results)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Wilmer\AppData\Local\Temp\tmpb3bfwzby.cplex.log' open.
CPLEX> Problem 'C:\Users\Wilmer\AppData\Local\Temp\tmppjse43j9.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : C:\Users\Wilmer\AppData\Local\Temp\tmppjse43j9.pyomo.lp
Objective sense      : Maximize
Variables            :      12  [General Integer: 12]
Objective nonzeros   :      12
Linear constraints   :       6  [Less: 3,  Greater: 3]
  Nonzeros           :      18
  RHS nonzeros       :       6

Variables            : Min LB: all infinite     Max UB: all infinite   
Objective nonzeros   : Min   